In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime

# Dublin的地理位置
dublin_lat = 53.35
dublin_lon = -6.26

# 使用Open-Meteo API获取Dublin当前天气和未来7天逐小时预测
url = f"https://api.open-meteo.com/v1/forecast?latitude={dublin_lat}&longitude={dublin_lon}&current=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,weather_code,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m&timezone=Europe/Dublin&forecast_days=14"

response = requests.get(url)
weather_data = response.json()

print("=" * 80)
print("Dublin当前天气信息:")
print("=" * 80)
if 'current' in weather_data:
    current = weather_data['current']
    print(f"\n当前时间: {current.get('time')}")
    print(f"温度: {current.get('temperature_2m')}°C")
    print(f"体感温度: {current.get('apparent_temperature')}°C")
    print(f"湿度: {current.get('relative_humidity_2m')}%")
    print(f"降水量: {current.get('precipitation')}mm")
    print(f"风速: {current.get('wind_speed_10m')}km/h")

# 处理未来7天小时预报数据
print("\n" + "=" * 80)
print("未来14天逐小时天气预报分析")
print("=" * 80)

if 'hourly' in weather_data:
    hourly = weather_data['hourly']
    times = hourly['time']
    temps = hourly['temperature_2m']
    humidity = hourly['relative_humidity_2m']
    precipitation = hourly['precipitation']
    wind_speed = hourly['wind_speed_10m']
    
    # 创建DataFrame用于分析
    forecast_df = pd.DataFrame({
        '时间': times,
        '温度(°C)': temps,
        '湿度(%)': humidity,
        '降水(mm)': precipitation,
        '风速(km/h)': wind_speed
    })
    
    # 找出风力最大的时间
    max_wind_idx = forecast_df['风速(km/h)'].idxmax()
    max_wind_time = forecast_df.loc[max_wind_idx, '时间']
    max_wind_speed = forecast_df.loc[max_wind_idx, '风速(km/h)']
    
    # 计算距离现在的小时数
    current_time = datetime.fromisoformat(current.get('time'))
    max_wind_datetime = datetime.fromisoformat(max_wind_time)
    hours_until_max_wind = int((max_wind_datetime - current_time).total_seconds() / 3600)
    
    print(f"\n【风力最强时刻】")
    print(f"时间: {max_wind_time}")
    print(f"风速: {max_wind_speed} km/h")
    print(f"距离现在: {hours_until_max_wind} 小时")
    
    # 显示风力最强时刻的完整信息
    print(f"\n【该时刻的详细信息】")
    print(f"温度: {forecast_df.loc[max_wind_idx, '温度(°C)']}°C")
    print(f"湿度: {forecast_df.loc[max_wind_idx, '湿度(%)']}%")
    print(f"降水: {forecast_df.loc[max_wind_idx, '降水(mm)']}mm")
    
    # 额外统计信息
    print("\n" + "=" * 80)
    print("14天风力统计摘要")
    print("=" * 80)
    print(f"最大风速: {max(wind_speed):.1f}km/h")
    print(f"最小风速: {min(wind_speed):.1f}km/h")
    print(f"平均风速: {sum(wind_speed)/len(wind_speed):.1f}km/h")
    
    # 显示Top 5风力最强的时间
    print("\n【未来14天风力Top 5】")
    top_5_wind = forecast_df.nlargest(5, '风速(km/h)')[['时间', '风速(km/h)']]
    for idx, (time, speed) in enumerate(zip(top_5_wind['时间'], top_5_wind['风速(km/h)']), 1):
        wind_datetime = datetime.fromisoformat(time)
        hours_diff = int((wind_datetime - current_time).total_seconds() / 3600)
        print(f"{idx}. {time} - {speed:.1f} km/h (距离现在 {hours_diff} 小时)")

天气预报 from 20260203T114126Z to 20260205T120108Z 每一个小时读取一次数据,存入json
api 来源是open weather, api=e6728031822d24b325d9661d5851f475

In [ ]:
import requests
import json
from datetime import datetime
from pathlib import Path

# Dublin坐标
dublin_lat = 53.35
dublin_lon = -6.26

# 定义时间范围（历史数据）
start_date = "2026-02-03"
end_date = "2026-02-05"

# 使用Open-Meteo API获取历史天气数据（完全免费，支持历史数据）
url = f"https://archive-api.open-meteo.com/v1/archive?latitude={dublin_lat}&longitude={dublin_lon}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m&timezone=Europe/Dublin"

print("=" * 80)
print(f"获取Dublin历史天气数据: {start_date} 至 {end_date}")
print("=" * 80)

try:
    response = requests.get(url, timeout=20)
    response.raise_for_status()
    weather_data = response.json()
    
    # 创建输出目录
    output_dir = Path("data/openweather_historical")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 保存完整JSON数据
    timestamp = datetime.now().strftime("%Y%m%dT%H%M%SZ")
    filename = output_dir / f"weather_historical_{start_date}_to_{end_date}_{timestamp}.json"
    
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(weather_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ 数据已保存: {filename}")
    print(f"✓ 文件大小: {filename.stat().st_size / 1024:.2f} KB")
    
    # 显示数据统计
    if 'hourly' in weather_data:
        hourly = weather_data['hourly']
        times = hourly['time']
        temps = hourly['temperature_2m']
        wind_speed = hourly['wind_speed_10m']
        
        print(f"\n数据统计:")
        print(f"  时间范围: {times[0]} 至 {times[-1]}")
        print(f"  数据点数: {len(times)} 小时")
        print(f"  温度范围: {min(temps):.1f}°C ~ {max(temps):.1f}°C")
        print(f"  平均风速: {sum(wind_speed)/len(wind_speed):.1f} km/h")
        print(f"  最大风速: {max(wind_speed):.1f} km/h")
    
    print("\n" + "=" * 80)
    print(f"数据保存位置: {output_dir}")
    print("=" * 80)
    
except Exception as e:
    print(f"✗ 错误: {e}")


In [ ]:
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

# 加载刚才保存的JSON数据
json_file = Path("data/openweather_historical/weather_historical_2026-02-03_to_2026-02-05_20260205T145631Z.json")

with open(json_file, "r", encoding="utf-8") as f:
    weather_data = json.load(f)

# WMO气象代码映射到中文
wmo_codes = {
    0: "晴朗", 1: "主要晴朗", 2: "部分多云", 3: "阴",
    45: "雾", 48: "沉积雾",
    51: "毛毛雨", 53: "中等毛毛雨", 55: "密集毛毛雨",
    61: "小雨", 63: "中雨", 65: "大雨",
    71: "小雪", 73: "中雪", 75: "大雪", 77: "雪粒",
    80: "小阵雨", 81: "中等阵雨", 82: "暴雨",
    85: "小阵雪", 86: "大阵雪",
    95: "雷暴", 96: "冰雹雷暴", 99: "大冰雹雷暴"
}

# 提取数据
hourly = weather_data['hourly']
df = pd.DataFrame({
    '时间': hourly['time'],
    '温度(°C)': hourly['temperature_2m'],
    '湿度(%)': hourly['relative_humidity_2m'],
    '降水(mm)': hourly['precipitation'],
    '天气代码': hourly['weather_code'],
})

# 添加中文天气描述
df['天气'] = df['天气代码'].map(lambda x: wmo_codes.get(x, f"未知({x})"))

print("=" * 100)
print("都柏林降水详细信息 (2026-02-03 至 2026-02-05)")
print("=" * 100)

# 显示有降水的时间
precipitation_events = df[df['降水(mm)'] > 0.0].copy()
print(f"\n【降水事件总数】: {len(precipitation_events)} 小时")
print(f"【总降水量】: {df['降水(mm)'].sum():.1f} mm")
print(f"【最大时均降水】: {df['降水(mm)'].max():.1f} mm\n")

print("降水时段列表：")
print("-" * 100)
for idx, row in precipitation_events.iterrows():
    print(f"{row['时间']} | 降水: {row['降水(mm)']:.1f} mm | 气温: {row['温度(°C)']:.1f}°C | 天气: {row['天气']}")

# 统计降水最强的时段
print("\n" + "=" * 100)
print("降水强度Top 10")
print("=" * 100)
top_10 = df.nlargest(10, '降水(mm)')[['时间', '降水(mm)', '天气', '温度(°C)']]
for idx, (i, row) in enumerate(top_10.iterrows(), 1):
    print(f"{idx:2d}. {row['时间']} | 降水: {row['降水(mm)']:.1f} mm | {row['天气']:6s} | 气温: {row['温度(°C)']:.1f}°C")

print("\n" + "=" * 100)


In [ ]:
import requests

all_articles = []
api_key = "37d41edc-7d44-4947-a1bf-e23550603052"

for page in range(1, 3):
    # 添加 show-fields=body 获取文章正文
    url = f"https://content.guardianapis.com/search?q=ireland&page-size=50&page={page}&show-fields=body,trailText&api-key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        results = data['response']['results']
        all_articles.extend(results)
        print(f"已获取第 {page} 页，共 {len(results)} 篇文章。")
    else:
        print(f"请求第 {page} 页时出错，状态码: {response.status_code}")

print(f"\n总共获取了 {len(all_articles)} 篇文章。\n")
for idx, article in enumerate(all_articles, 1):
    title = article['webTitle']
    # 优先获取bodyText，如果没有就用trailText
    body = article.get('fields', {}).get('bodyText', article.get('fields', {}).get('trailText', ''))[:100]
    publish_date = article.get('webPublicationDate', '未知日期')[:10]
    section = article.get('sectionName', '未分类')
    print(f"{idx}. {title}")
    print(f"发布日期: {publish_date} | 分类: {section}") 
    print(f"摘要:{body}")
    print("-" * 80)

In [ ]:
import requests
import folium
import pandas as pd

# 1. 获取实时数据 (JCDecaux)
api_key = "11c0b077ab18a5d4f761dc7b7469d89b7f5e22b3"
status_url = f"https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey={api_key}"

# 获取站点信息与实时状态 (同一接口返回)
stations = requests.get(status_url).json()

# 转为 DataFrame 方便处理
df = pd.DataFrame(stations)

# 2. 初始化地图 (以都柏林市中心为中心)
m = folium.Map(location=[53.3498, -6.2603], zoom_start=14, tiles='cartodbpositron')

# 3. 将站点添加到地图
for _, row in df.iterrows():
    # 定义颜色逻辑：没车红色，车少橙色，车多绿色
    available = row["available_bikes"]
    color = "green" if available > 5 else "orange" if available > 0 else "red"
    pos = row["position"]
    
    folium.CircleMarker(
        location=[pos["lat"], pos["lng"]],
        radius=7,
        popup=f"<b>{row['name']}</b><br>Available: {available}/{row['bike_stands']}",
        color=color,
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 4. 在 Jupyter 中直接显示地图
m


In [ ]:
import json
import time
from datetime import datetime, timezone, timedelta
from pathlib import Path

# 每5分钟抓取一次，保存为独立 JSON 文件；出错不影响下一轮
api_key = "11c0b077ab18a5d4f761dc7b7469d89b7f5e22b3"
status_url = f"https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey={api_key}"
output_dir = Path("data/dublinbike_status")
output_dir.mkdir(parents=True, exist_ok=True)

def fetch_and_save_once():
    ts = datetime.now(timezone.utc)
    ts_str = ts.strftime("%Y%m%dT%H%M%SZ")
    filename = output_dir / f"station_status_{ts_str}.json"
    try:
        resp = requests.get(status_url, timeout=20)
        resp.raise_for_status()
        data = resp.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)
        print(f"saved: {filename}")
    except Exception as e:
        print(f"error: {e}; will retry next cycle")

# 运行48小时自动停止
max_hours = 48
end_time = datetime.now(timezone.utc) + timedelta(hours=max_hours)

# 运行时可按下停止按钮中断
while datetime.now(timezone.utc) < end_time:
    fetch_and_save_once()
    time.sleep(300)
print("done: reached 48h timeout")


In [1]:
import dbinfo
import requests
# JCKEY = "11c0b077ab18a5d4f761dc7b7469d89b7f5e22b3"
# NAME = "dublin"
# STATIONS_URI = "https://api.jcdecaux.com/vls/v1/stations/42"
import json
r = requests.get(dbinfo.STATIONS_URI, params={"apiKey": dbinfo.JCKEY,
"contract": dbinfo.NAME})
data = json.loads(r.text)
print(json.dumps(data, indent=4))

{
    "number": 25,
    "contract_name": "dublin",
    "name": "MERRION SQUARE EAST",
    "address": "Merrion Square East",
    "position": {
        "lat": 53.339069,
        "lng": -6.247006
    },
    "banking": false,
    "bonus": false,
    "bike_stands": 30,
    "available_bike_stands": 23,
    "available_bikes": 7,
    "status": "OPEN",
    "last_update": 1770308776000
}
